<a href="https://colab.research.google.com/github/ekvirika/WalmartRecruiting/blob/main/notebooks/model_experiment_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:

# Install required packages
!pip install -q wandb torch torchvision pandas numpy matplotlib seaborn scikit-learn mlflow pytorch_lightning pytorch_forecasting

# Set up Kaggle API
!pip install -q kaggle  dagshub

In [45]:

# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json


In [46]:

# Download the dataset
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -q walmart-recruiting-store-sales-forecasting.zip

walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
replace features.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [47]:

!unzip -q train.csv.zip
!unzip -q stores.csv.zip
!unzip -q test.csv.zip
!unzip -q features.csv.zip


replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
unzip:  cannot find or open stores.csv.zip, stores.csv.zip.zip or stores.csv.zip.ZIP.
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [48]:
!pip install dagshub mlflow --quiet
! pip install optuna

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin

import lightgbm as lgb
import mlflow
import mlflow.lightgbm
from mlflow.tracking import MlflowClient

warnings.filterwarnings('ignore')


In [50]:
import mlflow
from dagshub import dagshub_logger
import os

# Set tracking URI manually
mlflow.set_tracking_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

# Use your DagsHub credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "ekvirika"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "0adb1004ddd4221395353efea2d8ead625e26197"

# Optional: set registry if you're using model registry
mlflow.set_registry_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

In [51]:
# import mlflow.sklearn
# import mlflow.lightgbm
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from lightgbm import LGBMRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import RandomizedSearchCV
# import matplotlib.pyplot as plt
# import joblib




# # ----------------------
# # Evaluation Metric (WMAE)
# # ----------------------
# def weighted_mae(y_true, y_pred, is_holiday):
#     weights = np.where(is_holiday, 5, 1)
#     return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

# # ----------------------
# # Data Loader
# # ----------------------
# def load_data():
#     with mlflow.start_run(run_name="LGBM_Data_Loading", nested=True):
#         train_df = pd.read_csv('train.csv')
#         test_df = pd.read_csv('test.csv')
#         stores_df = pd.read_csv('stores.csv')
#         features_df = pd.read_csv('features.csv')

#         df = train_df.merge(features_df, on=['Store', 'Date'], how='inner') \
#                      .merge(stores_df, on='Store', how='inner')

#         mlflow.log_params({
#             "train_shape": train_df.shape,
#             "test_shape": test_df.shape,
#             "missing_values_train": train_df.isnull().sum().sum(),
#             "missing_values_test": test_df.isnull().sum().sum()
#         })

#         return df

# # ----------------------
# # Preprocessing
# # ----------------------
# def preprocess(df):
#     with mlflow.start_run(run_name="LGBM_Cleaning", nested=True):


#       df = df.copy()

#       # Fix column naming issues
#       df["IsHoliday"] = df.pop("IsHoliday_x") if "IsHoliday_x" in df else df["IsHoliday"]
#       df.drop(columns=["IsHoliday_y"], errors='ignore', inplace=True)

#       # Filter and sort
#       df = df[df["Weekly_Sales"] > 0]
#       df["Date"] = pd.to_datetime(df["Date"])
#       df = df.sort_values(by="Date")

#       # Remove outliers
#       q_low = df["Weekly_Sales"].quantile(0.01)
#       q_high = df["Weekly_Sales"].quantile(0.99)
#       df = df[(df["Weekly_Sales"] >= q_low) & (df["Weekly_Sales"] <= q_high)]

#       # Lag features
#       for lag in [1, 2, 4, 52]:
#           df[f"lag_{lag}"] = df.groupby("Store")["Weekly_Sales"].shift(lag)

#       # Rolling statistics
#       df["rolling_mean_4"] = df.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).mean()
#       df["rolling_std_4"] = df.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).std()

#       # Fill missing MarkDowns
#       markdown_cols = [col for col in df.columns if "MarkDown" in col]
#       df[markdown_cols] = df[markdown_cols].fillna(0)

#       # Date features
#       df["Year"] = df["Date"].dt.year
#       df["Month"] = df["Date"].dt.month
#       df["Week"] = df["Date"].dt.isocalendar().week
#       df["DayOfWeek"] = df["Date"].dt.dayofweek
#       df["IsMonthStart"] = df["Date"].dt.is_month_start.astype(int)
#       df["IsMonthEnd"] = df["Date"].dt.is_month_end.astype(int)
#       df["Quarter"] = df["Date"].dt.quarter
#       df.drop(columns=["Date"], inplace=True)

#       mlflow.log_params({"droppped_cols": "IsHoliday_x, IsHoliday_y, Date", "fill_markdown_NaNs": "0"})

#       # Drop rows with missing lag/rolling values
#       df = df.dropna()

#       return df

# # ----------------------
# # Dynamic Pipeline Builder
# # ----------------------
# def build_pipeline(X, model=None):
#     with mlflow.start_run(run_name="LGBM_Feature_Engineering", nested=True):

#       numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
#       categorical_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

#       preprocessor = ColumnTransformer(transformers=[
#           ("num", SimpleImputer(strategy='mean'), numeric_cols),
#           ("cat", OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
#       ])

#       if model is None:
#           model = LGBMRegressor(
#               n_estimators=100,
#               max_depth=6,
#               learning_rate=0.1,
#               random_state=42,
#               n_jobs=-1
#           )

#       pipeline = Pipeline([
#           ("preprocessor", preprocessor),
#           ("model", model)
#       ])

#       mlflow.log_params({"num_cols": numeric_cols, "cat_cols": categorical_cols,
#                         "simple_imputer": "mean", "cat_encoder": "one_hot", "pipeline": pipeline})

#       return pipeline

# # ----------------------
# # Runner
# # ----------------------
# def run():
#     # with mlflow.start_run(run_name="XGBoost_Training"):

#       df = load_data()
#       df = preprocess(df)

#       # Define features and target
#       X = df.drop(columns=["Weekly_Sales"])
#       y = df["Weekly_Sales"]

#       # Split
#       X_train, X_test, y_train, y_test = train_test_split(
#           X, y, test_size=0.2, random_state=42
#       )

#       # Store IsHoliday for WMAE
#       is_holiday_test = X_test["IsHoliday"].astype(bool).values if "IsHoliday" in X_test.columns else np.zeros_like(y_test)

#       # Train
#       pipeline = build_pipeline(X_train)

#       # Hyperparameter search space
#       param_dist = {
#           "model__n_estimators": [100, 200, 300, 500, 800, 1000],
#           "model__max_depth": [3, 6, 9, -1],  # -1 means no limit
#           "model__learning_rate": [0.01, 0.1],
#           "model__subsample": [0.6, 0.8, 1.0],  # still valid
#           "model__colsample_bytree": [0.6, 0.8, 1.0],  # also valid
#           "model__num_leaves": [31, 63, 127, 255, 511]
#       }


#       search = RandomizedSearchCV(
#           pipeline,
#           param_distributions=param_dist,
#           n_iter=20,
#           cv=3,
#           scoring="neg_mean_absolute_error",  # you could also define custom scorer using WMAE if needed
#           verbose=2,
#           n_jobs=-1,
#           random_state=42
#       )

#       search.fit(X_train, y_train)

#       best_model = search.best_estimator_
#       y_pred = best_model.predict(X_test)

#       # WMAE
#       wmae_score = weighted_mae(y_test.values, y_pred, is_holiday_test)
#       print(f"✅ WMAE: {wmae_score:.2f}")

#       mlflow.log_params(search.best_params_)
#       mlflow.log_metric("WMAE", wmae_score)
#       # mlflow.sklearn.log_model(best_model.named_steps["model"], artifact_path="lgbm_model")
#       joblib.dump(best_model.named_steps["model"], "lgbm_model.pkl")
#       mlflow.log_artifact("lgbm_model.pkl")

#       joblib.dump(best_model, "lgbm_pipeline.pkl")
#       mlflow.log_artifact("lgbm_pipeline.pkl")


#       # Save feature importance plot as artifact
#       fig, ax = plt.subplots()
#       lgb_model = best_model.named_steps["model"]
#       importances = lgb_model.feature_importances_
#       ax.bar(range(len(importances)), importances)
#       plt.savefig("feature_importance.png")
#       mlflow.log_artifact("feature_importance.png")


#       return best_model

# # ----------------------
# # Entry Point
# # ----------------------
# if __name__ == "__main__":
#     try:
#         with mlflow.start_run(run_name="LGBM_Training"):
#             model = run()
#         print("✅ Pipeline executed successfully!")
#     except Exception as e:
#         print(f"❌ Pipeline failed: {e}")


In [52]:
# import pandas as pd
# import numpy as np
# import mlflow
# import mlflow.sklearn
# import mlflow.lightgbm
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from lightgbm import LGBMRegressor
# from sklearn.metrics import mean_absolute_error
# import matplotlib.pyplot as plt
# import joblib

# # ----------------------
# # Evaluation Metric (WMAE)
# # ----------------------
# def weighted_mae(y_true, y_pred, is_holiday):
#     weights = np.where(is_holiday, 5, 1)
#     return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

# # ----------------------
# # Custom Transformers
# # ----------------------
# class DataMerger(BaseEstimator, TransformerMixin):
#     def __init__(self, features_df, stores_df):
#         self.features_df = features_df
#         self.stores_df = stores_df

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         X = X.copy()
#         X = X.merge(self.features_df, on=['Store', 'Date'], how='inner')
#         X = X.merge(self.stores_df, on='Store', how='inner')
#         return X

# class DataCleaner(BaseEstimator, TransformerMixin):
#     def __init__(self, is_train=True):
#         self.q_low = None
#         self.q_high = None
#         self.is_train = is_train

#     def fit(self, X, y=None):
#         if self.is_train and 'Weekly_Sales' in X.columns:
#             self.q_low = X["Weekly_Sales"].quantile(0.01)
#             self.q_high = X["Weekly_Sales"].quantile(0.99)
#         return self

#     def transform(self, X):
#         X = X.copy()

#         if "IsHoliday_x" in X.columns:
#             X["IsHoliday"] = X.pop("IsHoliday_x")
#         X.drop(columns=["IsHoliday_y"], errors='ignore', inplace=True)

#         X["Date"] = pd.to_datetime(X["Date"])
#         X = X.sort_values(by="Date")

#         if self.is_train and 'Weekly_Sales' in X.columns:
#             X = X[X["Weekly_Sales"] > 0]
#             if self.q_low is not None and self.q_high is not None:
#                 X = X[(X["Weekly_Sales"] >= self.q_low) & (X["Weekly_Sales"] <= self.q_high)]

#         return X


# class FeatureEngineer(BaseEstimator, TransformerMixin):
#     def __init__(self, is_train=True):
#         self.is_train = is_train

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         X = X.copy()

#         if self.is_train and 'Weekly_Sales' in X.columns:
#             for lag in [1, 2, 4, 52]:
#                 X[f"lag_{lag}"] = X.groupby("Store")["Weekly_Sales"].shift(lag)
#             X["rolling_mean_4"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).mean()
#             X["rolling_std_4"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).std()

#         markdown_cols = [col for col in X.columns if "MarkDown" in col]
#         X[markdown_cols] = X[markdown_cols].fillna(0)

#         X["Year"] = X["Date"].dt.year
#         X["Month"] = X["Date"].dt.month
#         X["Week"] = X["Date"].dt.isocalendar().week
#         X["DayOfWeek"] = X["Date"].dt.dayofweek
#         X["IsMonthStart"] = X["Date"].dt.is_month_start.astype(int)
#         X["IsMonthEnd"] = X["Date"].dt.is_month_end.astype(int)
#         X["Quarter"] = X["Date"].dt.quarter

#         X.drop(columns=["Date"], inplace=True)
#         return X


# class MissingValueHandler(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         self.fill_values = {}
#         pass

#     def fit(self, X, y=None):
#         # Store mean values for numerical columns
#         numeric_cols = X.select_dtypes(include=[np.number]).columns
#         for col in numeric_cols:
#             if X[col].isnull().any():
#                 self.fill_values[col] = X[col].mean()
#         return self

#     def transform(self, X):
#         X = X.copy()

#         # Fill numerical columns with stored means
#         for col, fill_value in self.fill_values.items():
#             if col in X.columns:
#                 X[col] = X[col].fillna(fill_value)

#         # Drop rows with any remaining missing values (from lag features)
#         X = X.dropna()

#         return X

# class FeatureSelector(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         self.numeric_cols = []
#         self.categorical_cols = []

#     def fit(self, X, y=None):
#         self.numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
#         self.categorical_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
#         return self

#     def transform(self, X):
#         # This transformer just passes through the data but stores column info
#         return X

#     def get_feature_names_out(self):
#         return self.numeric_cols, self.categorical_cols

In [53]:
# # from sklearn.pipeline import Pipeline
# # from lightgbm import LGBMRegressor

# # full_pipeline = Pipeline([
# #     ('merger', DataMerger(features_df, stores_df)),
# #     ('cleaner', DataCleaner()),
# #     ('feature_engineer', FeatureEngineer()),
# #     ('missing_handler', MissingValueHandler()),
# #     ('feature_selector', FeatureSelector()),
# #     ('model', LGBMRegressor(
# #         n_estimators=800,
# #         max_depth=-1,
# #         learning_rate=0.1,
# #         random_state=42,
# #         n_jobs=-1,
# #         num_leaves=63
# #     ))
# # ])

# # Define preprocessing after FeatureSelector
# preprocessor = ColumnTransformer(
#     transformers=[
#         ("num", "passthrough", []),  # to be filled after fit
#         ("cat", OneHotEncoder(handle_unknown="ignore"), [])
#     ]
# )

# # Full pipeline with preprocessing + model
# full_pipeline = Pipeline([
#     ('merger', DataMerger(features_df, stores_df)),
#     ('cleaner', DataCleaner(is_train=False)),
#     ('feature_engineer', FeatureEngineer(is_train=False)),
#     ('missing_handler', MissingValueHandler()),
#     ('feature_selector', FeatureSelector()),  # No transform here
#     ('preprocessor', preprocessor),          # You’ll configure this dynamically
#     ('model', LGBMRegressor(
#         n_estimators=800,
#         max_depth=-1,
#         learning_rate=0.1,
#         random_state=42,
#         n_jobs=-1,
#         num_leaves=63
#     ))
# ])


In [54]:
# # ----------------------
# # Training and Results
# # ----------------------
# def train_and_evaluate(full_pipeline, train_df, test_df, features_df, stores_df):
#     """Complete training pipeline with results"""

#     with mlflow.start_run(run_name="Walmart_Pipeline_Training"):

#         print("🔄 Starting pipeline training...")

#         # Step 1: Process all training data first
#         temp_pipeline = Pipeline([
#             ('merger', DataMerger(features_df, stores_df)),
#             ('cleaner', DataCleaner()),
#             ('feature_engineer', FeatureEngineer()),
#             ('missing_handler', MissingValueHandler()),
#             ('feature_selector', FeatureSelector())
#         ])

#         # Get processed data
#         X_processed = temp_pipeline.fit_transform(train_df)

#         # Configure preprocessor columns
#         numeric_cols = X_processed.select_dtypes(include=[np.number]).columns.tolist()
#         categorical_cols = X_processed.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

#         # Remove target from numeric columns if present
#         if 'Weekly_Sales' in numeric_cols:
#             numeric_cols.remove('Weekly_Sales')

#         # Update preprocessor
#         full_pipeline.named_steps['preprocessor'].transformers = [
#             ("num", "passthrough", numeric_cols),
#             ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
#         ]

#         print(f"📈 Numeric columns: {len(numeric_cols)}")
#         print(f"📊 Categorical columns: {len(categorical_cols)}")

#         # Step 2: Separate features and target from processed data
#         X_features = X_processed.drop(columns=["Weekly_Sales"]) if "Weekly_Sales" in X_processed.columns else X_processed
#         y = X_processed["Weekly_Sales"] if "Weekly_Sales" in X_processed.columns else None

#         print(f"🔄 Total processed samples: {len(X_features)}")

#         # Step 3: Train-test split on processed data
#         X_train, X_test, y_train, y_test = train_test_split(
#             X_features, y, test_size=0.2, random_state=42
#         )

#         print(f"🔄 Training set size: {len(X_train)}")
#         print(f"🔄 Test set size: {len(X_test)}")

#         # Step 4: Create a simplified pipeline for final preprocessing + model
#         # Since we already did most preprocessing, we just need the final steps
#         final_pipeline = Pipeline([
#             ('preprocessor', full_pipeline.named_steps['preprocessor']),
#             ('model', full_pipeline.named_steps['model'])
#         ])

#         # Step 5: Fit the final pipeline
#         print("🚀 Training final pipeline...")
#         final_pipeline.fit(X_train, y_train)

#         # Step 6: Make predictions
#         print("🔮 Making predictions...")
#         y_pred_train = final_pipeline.predict(X_train)
#         y_pred_test = final_pipeline.predict(X_test)

#         # test = pd.read_csv('test.csv')

#         # test["Date"] = pd.to_datetime(test["Date"]).dt.strftime("%Y-%m-%d")

#         # test["Id"] = test["Store"].astype(str) + "_" + test["Dept"].astype(str) + "_" + test["Date"]

#         # submission = test[["Id"]].copy()
#         # submission["Weekly_Sales"] = y_pred_test

#         # submission.to_csv("submission.csv", index=False)


#         # Step 7: Get IsHoliday for WMAE calculation
#         is_holiday_test = X_test["IsHoliday"].astype(bool).values if "IsHoliday" in X_test.columns else np.zeros_like(y_test, dtype=bool)

#         # Step 8: Calculate metrics
#         train_wmae = weighted_mae(y_train.values, y_pred_train, np.zeros_like(y_train, dtype=bool))
#         test_wmae = weighted_mae(y_test.values, y_pred_test, is_holiday_test)

#         train_mae = mean_absolute_error(y_train, y_pred_train)
#         test_mae = mean_absolute_error(y_test, y_pred_test)

#         # Step 9: Print results
#         print("\n" + "="*50)
#         print("📊 TRAINING RESULTS")
#         print("="*50)
#         print(f"🎯 Training WMAE: {train_wmae:.2f}")
#         print(f"🎯 Test WMAE: {test_wmae:.2f}")
#         print(f"📈 Training MAE: {train_mae:.2f}")
#         print(f"📈 Test MAE: {test_mae:.2f}")
#         print(f"🔍 Holiday samples in test: {is_holiday_test.sum()}")
#         print("="*50)

#         # Step 10: Log to MLflow
#         mlflow.log_params({
#             "model_type": "LGBMRegressor",
#             "n_estimators": 800,
#             "max_depth": -1,
#             "learning_rate": 0.1,
#             "num_leaves": 63,
#             "train_size": len(X_train),
#             "test_size": len(X_test),
#             "numeric_features": len(numeric_cols),
#             "categorical_features": len(categorical_cols)
#         })

#         mlflow.log_metrics({
#             "train_wmae": train_wmae,
#             "test_wmae": test_wmae,
#             "train_mae": train_mae,
#             "test_mae": test_mae
#         })

#         # Step 11: Feature importance
#         lgb_model = final_pipeline.named_steps['model']
#         feature_names = [f"feature_{i}" for i in range(len(lgb_model.feature_importances_))]

#         importance_df = pd.DataFrame({
#             'feature': feature_names,
#             'importance': lgb_model.feature_importances_
#         }).sort_values('importance', ascending=False)

#         print(f"\n🔝 Top 10 Most Important Features:")
#         print("-" * 40)
#         for i, (_, row) in enumerate(importance_df.head(10).iterrows()):
#             print(f"{i+1:2d}. {row['feature']:<20} {row['importance']:.4f}")

#         # Step 12: Create complete pipeline for saving/prediction
#         complete_pipeline = Pipeline([
#             ('preprocessing', temp_pipeline),
#             ('final_model', final_pipeline)
#         ])

#         # Step 13: Save models
#         joblib.dump(complete_pipeline, "walmart_complete_pipeline.pkl")
#         mlflow.log_artifact("walmart_complete_pipeline.pkl")

#         # mlflow.sklearn.log_model(
#         #     complete_pipeline,
#         #     artifact_path="complete_pipeline",
#         #     registered_model_name="walmart_sales_pipeline"
#         # )
#         # mlflow.sklearn.log_model(complete_pipeline, artifact_path="pipeline")
#         joblib.dump(complete_pipeline, "walmart_sales_pipeline")

#         print("✅ Training completed successfully!")
#         print(f"🎯 Final Test WMAE: {test_wmae:.2f}")

#         return complete_pipeline, {
#             'train_wmae': train_wmae,
#             'test_wmae': test_wmae,
#             'train_mae': train_mae,
#             'test_mae': test_mae
#         }

# # def make_test_predictions(pipeline, test_df):
# #     """Make predictions on test data"""
# #     print("🔮 Making predictions on test data...")

# #     predictions = pipeline.predict(test_df)

# #     submission_df = pd.DataFrame({
# #         'Id': test_df['Id'] if 'Id' in test_df.columns else range(len(predictions)),
# #         'Weekly_Sales': predictions
# #     })

# #     submission_df.to_csv("walmart_submission.csv", index=False)

# #     print(f"📊 Generated {len(predictions)} predictions")
# #     print(f"📊 Prediction range: {predictions.min():.2f} to {predictions.max():.2f}")
# #     print("💾 Submission saved as 'walmart_submission.csv'")

# #     return submission_df

# # def make_test_predictions(pipeline, test_df):
# #     """Make predictions on test data"""
# #     print("🔮 Making predictions on test data...")

# #     # Load required files for merging and historical context
# #     stores_df = pd.read_csv("stores.csv")
# #     features_df = pd.read_csv("features.csv")
# #     train_df = pd.read_csv("train.csv")

# #     # Concatenate train and test for proper lag/rolling feature computation
# #     combined_df = pd.concat([train_df, test_df], sort=False)
# #     combined_df.sort_values(["Store", "Dept", "Date"], inplace=True)

# #     # Mark test rows
# #     combined_df["is_test"] = combined_df["Weekly_Sales"].isna()

# #     # Apply only preprocessing steps (must be same order as in training)
# #     temp_pipeline = Pipeline([
# #         ('merger', DataMerger(features_df, stores_df)),
# #         ('cleaner', DataCleaner()),
# #         ('feature_engineer', FeatureEngineer()),
# #         ('missing_handler', MissingValueHandler()),
# #         ('feature_selector', FeatureSelector())
# #     ])

# #     full_processed = temp_pipeline.fit_transform(combined_df)

# #     # Separate test data after transformation
# #     test_processed = full_processed[full_processed["is_test"] == True].drop(columns=["is_test", "Weekly_Sales"], errors='ignore')

# #     # Drop Id if needed
# #     ids = test_processed["Id"] if "Id" in test_processed.columns else range(len(test_processed))
# #     if "Id" in test_processed.columns:
# #         test_processed = test_processed.drop(columns=["Id"])

# #     # Get the final model step
# #     model_pipeline = pipeline.named_steps["final_model"]

# #     # Make predictions
# #     predictions = model_pipeline.predict(test_processed)

# #     submission_df = pd.DataFrame({
# #         "Id": ids,
# #         "Weekly_Sales": predictions
# #     })

# #     submission_df.to_csv("walmart_submission.csv", index=False)

# #     print(f"📊 Generated {len(predictions)} predictions")
# #     print(f"📊 Prediction range: {predictions.min():.2f} to {predictions.max():.2f}")
# #     print("💾 Submission saved as 'walmart_submission.csv'")

# #     return submission_df


# # ----------------------
# # Main execution
# # ----------------------
# if __name__ == "__main__":
#     try:
#         # Load data
#         train_df = pd.read_csv('train.csv')
#         test_df = pd.read_csv('test.csv')
#         stores_df = pd.read_csv('stores.csv')
#         features_df = pd.read_csv('features.csv')

#         print("🚀 Starting Walmart Sales Forecasting Pipeline")
#         print("=" * 60)

#         # Train the model
#         pipeline, results = train_and_evaluate(full_pipeline, train_df, test_df, features_df, stores_df)

#         # # Make test predictions
#         # print("\n" + "="*50)
#         # print("📝 MAKING TEST PREDICTIONS")
#         # print("="*50)

#         # submission_df = make_test_predictions(pipeline, test_df)

#         # Final summary
#         print("\n" + "="*60)
#         print("🎉 PIPELINE EXECUTION COMPLETE!")
#         print("="*60)
#         print(f"✅ Best Test WMAE: {results['test_wmae']:.2f}")
#         # print(f"✅ Best Test MAE: {results['test_mae']:.2f}")
#         print(f"💾 Pipeline saved: walmart_complete_pipeline.pkl")
#         # print(f"📝 Submission file: walmart_submission.csv")
#         print("🎯 Ready for Kaggle submission!")

#     except Exception as e:
#         print(f"❌ Pipeline failed: {e}")
#         raise

In [160]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
features = pd.read_csv('features.csv')

In [167]:
# from sklearn.base import BaseEstimator, TransformerMixin

# class BaseMerger(BaseEstimator, TransformerMixin):

#     def __init__(self, features_df, stores_df):
#         self.features_df = features_df.copy()
#         self.stores_df = stores_df.copy()
#         self.features_df["Date"] = pd.to_datetime(self.features_df["Date"])
#         self.stores_df["Store"] = self.stores_df["Store"].astype(int)

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         X = X.copy()
#         X["Date"] = pd.to_datetime(X["Date"])
#         X["Store"] = X["Store"].astype(int)
#         merged = X.merge(self.features_df, on=["Store", "Date"], how="left")
#         merged = merged.merge(self.stores_df, on="Store", how="left")
#         return merged

# # class FeatureAdder(BaseEstimator, TransformerMixin):

# #     def __init__(self, is_train=True):
# #         self.is_train = is_train

# #     def fit(self, X, y=None):
# #         return self

# #     def transform(self, X):
# #         X = X.copy()
# #         X["Date"] = pd.to_datetime(X["Date"], errors="coerce")

# #         if self.is_train and "Weekly_Sales" in X.columns:
# #             for lag in [1, 2, 4, 52]:
# #                 X[f"lag_{lag}"] = X.groupby("Store")["Weekly_Sales"].shift(lag)
# #             X["rolling_mean_4"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(4).mean()
# #             X["rolling_std_4"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(4).std()

# #         markdown_cols = [col for col in X.columns if "MarkDown" in col]
# #         X[markdown_cols] = X[markdown_cols].fillna(0)

# #         X["Year"] = X["Date"].dt.year
# #         X["Month"] = X["Date"].dt.month
# #         X["Week"] = X["Date"].dt.isocalendar().week
# #         X["DayOfWeek"] = X["Date"].dt.dayofweek
# #         X["IsMonthStart"] = X["Date"].dt.is_month_start.astype(int)
# #         X["IsMonthEnd"] = X["Date"].dt.is_month_end.astype(int)
# #         X["Quarter"] = X["Date"].dt.quarter

# #         # ⚠️ Don't drop Date here — it's needed for aggregations!
# #         return X

# class FeatureAdder(BaseEstimator, TransformerMixin):
#     def __init__(self, is_train=True, use_log_sales=False):
#         self.is_train = is_train
#         self.use_log_sales = use_log_sales

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         X = X.copy()
#         X["Date"] = pd.to_datetime(X["Date"], errors="coerce")

#         if self.is_train and "Weekly_Sales" in X.columns:
#             if self.use_log_sales:
#                 X["Weekly_Sales"] = np.log1p(X["Weekly_Sales"])

#             # Lag features per Store + Dept
#             for lag in [1, 2, 4, 52]:
#                 X[f"lag_{lag}"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(lag)

#             # Rolling features
#             X["rolling_mean_4"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1).rolling(4).mean()
#             X["rolling_std_4"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1).rolling(4).std()

#         # Time-related features
#         X["Year"] = X["Date"].dt.year
#         X["Month"] = X["Date"].dt.month
#         X["Week"] = X["Date"].dt.isocalendar().week
#         X["DayOfWeek"] = X["Date"].dt.dayofweek
#         X["IsMonthStart"] = X["Date"].dt.is_month_start.astype(int)
#         X["IsMonthEnd"] = X["Date"].dt.is_month_end.astype(int)
#         X["Quarter"] = X["Date"].dt.quarter
#         X["IsWeekend"] = X["DayOfWeek"].isin([5, 6]).astype(int)

#         # Seasonality
#         X["sin_week"] = np.sin(2 * np.pi * X["Week"] / 52)
#         X["cos_week"] = np.cos(2 * np.pi * X["Week"] / 52)

#         markdown_cols = [col for col in X.columns if "MarkDown" in col]
#         X[markdown_cols] = X[markdown_cols].fillna(0)

#         return X


# class MissingValueFiller(BaseEstimator, TransformerMixin):

#     def __init__(self, is_train=True):
#         self.q_low = None
#         self.q_high = None
#         self.is_train = is_train

#     def fit(self, X, y=None):
#         if self.is_train and "Weekly_Sales" in X.columns:
#             self.q_low = X["Weekly_Sales"].quantile(0.01)
#             self.q_high = X["Weekly_Sales"].quantile(0.99)
#         return self

#     def transform(self, X):
#         X = X.copy()
#         X["Date"] = pd.to_datetime(X["Date"])

#         if self.is_train and "Weekly_Sales" in X.columns:
#             X = X[X["Weekly_Sales"] > 0]
#             if self.q_low is not None and self.q_high is not None:
#                 X = X[(X["Weekly_Sales"] >= self.q_low) & (X["Weekly_Sales"] <= self.q_high)]

#         return X


# class CategoricalEncoder(BaseEstimator, TransformerMixin):

#     def __init__(self):
#         self.fill_values = {}

#     def fit(self, X, y=None):
#         numeric_cols = X.select_dtypes(include=[np.number]).columns
#         for col in numeric_cols:
#             if X[col].isnull().any():
#                 self.fill_values[col] = X[col].mean()
#         return self

#     def transform(self, X):
#         X = X.copy()

#         for col, fill_value in self.fill_values.items():
#             if col in X.columns:
#                 X[col] = X[col].fillna(fill_value)

#         # if "IsHoliday" in X.columns:
#         #     X["IsHoliday"] = X["IsHoliday"].astype(int)

#         # X = X.dropna()  # Now okay, since lags are filled

#         if "IsHoliday" in X.columns:
#             X["IsHoliday"] = X["IsHoliday"].fillna(0).astype(int)  # Fill missing with 0 (not holiday)

#         if "Type" in X.columns:
#             X["Type"] = X["Type"].map({"A": 3, "B": 2, "C": 1}).fillna(0)

#         # Fill missing values for numeric columns as before
#         for col, fill_value in self.fill_values.items():
#             if col in X.columns:
#                 X[col] = X[col].fillna(fill_value)

#         # Now drop rows only if they still have missing values in important columns (exclude 'IsHoliday')
#         X = X.dropna(subset=[col for col in X.columns if col != "IsHoliday"])

#         return X

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin


class BaseMerger(BaseEstimator, TransformerMixin):
    def __init__(self, features_df, stores_df):
        self.features_df = features_df.copy()
        self.stores_df = stores_df.copy()
        self.features_df["Date"] = pd.to_datetime(self.features_df["Date"])
        self.stores_df["Store"] = self.stores_df["Store"].astype(int)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"])
        X["Store"] = X["Store"].astype(int)
        merged = X.merge(self.features_df, on=["Store", "Date"], how="left")
        merged = merged.merge(self.stores_df, on="Store", how="left")
        return merged


class MissingValueFiller(BaseEstimator, TransformerMixin):
    def __init__(self, is_train=True):
        self.q_low = None
        self.q_high = None
        self.is_train = is_train

    def fit(self, X, y=None):
        if self.is_train and "Weekly_Sales" in X.columns:
            self.q_low = X["Weekly_Sales"].quantile(0.01)
            self.q_high = X["Weekly_Sales"].quantile(0.99)
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"])

        if self.is_train and "Weekly_Sales" in X.columns:
            X = X[X["Weekly_Sales"] > 0]
            if self.q_low is not None and self.q_high is not None:
                X = X[(X["Weekly_Sales"] >= self.q_low) & (X["Weekly_Sales"] <= self.q_high)]

        return X


class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.fill_values = {}

    def fit(self, X, y=None):
        numeric_cols = X.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if X[col].isnull().any():
                self.fill_values[col] = X[col].mean()
        return self

    def transform(self, X):
        X = X.copy()

        for col, fill_value in self.fill_values.items():
            if col in X.columns:
                X[col] = X[col].fillna(fill_value)

        if "IsHoliday" in X.columns:
            X["IsHoliday"] = X["IsHoliday"].fillna(0).astype(int)

        if "Type" in X.columns:
            X["Type"] = X["Type"].map({"A": 3, "B": 2, "C": 1}).fillna(0)

        X = X.dropna(subset=[col for col in X.columns if col != "IsHoliday"])

        return X


class FeatureAdder(BaseEstimator, TransformerMixin):
    def __init__(self, is_train=True):
        self.is_train = is_train

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"], errors="coerce")

        if self.is_train and "Weekly_Sales" in X.columns:
            for lag in [1, 2, 4, 52]:
                X[f"lag_{lag}"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(lag)
            X["rolling_mean_4"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1).rolling(4).mean()
            X["rolling_std_4"] = X.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1).rolling(4).std()

        markdown_cols = [col for col in X.columns if "MarkDown" in col]
        X[markdown_cols] = X[markdown_cols].fillna(0)

        X["Year"] = X["Date"].dt.year
        X["Month"] = X["Date"].dt.month
        X["Week"] = X["Date"].dt.isocalendar().week
        X["DayOfWeek"] = X["Date"].dt.dayofweek
        X["IsMonthStart"] = X["Date"].dt.is_month_start.astype(int)
        X["IsMonthEnd"] = X["Date"].dt.is_month_end.astype(int)
        X["Quarter"] = X["Date"].dt.quarter

        # Interaction terms
        if "Size" in X.columns and "Fuel_Price" in X.columns:
            X["Size_x_Fuel"] = X["Size"] * X["Fuel_Price"]

        if "CPI" in X.columns and "Unemployment" in X.columns:
            X["CPI_Unemployment"] = X["CPI"] * X["Unemployment"]

        return X


class DeptProportionAdder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        if 'Weekly_Sales' in X.columns:
            self.proportions = (
                X.groupby(['Store', 'Dept'])['Weekly_Sales'].sum() /
                X.groupby('Store')['Weekly_Sales'].sum()
            ).fillna(0).to_dict()
        else:
            self.proportions = {}
        return self

    def transform(self, X):
        X = X.copy()
        X['Dept_Proportion'] = X.apply(
            lambda row: self.proportions.get((row['Store'], row['Dept']), 0.0),
            axis=1
        )
        return X


class LagSalesAdder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.sort_values(['Store', 'Dept', 'Date']).copy()
        group = X.groupby(['Store', 'Dept'])
        X['Sales_Lag_1'] = group['Weekly_Sales'].shift(1).fillna(0)
        X['Sales_Lag_2'] = group['Weekly_Sales'].shift(2).fillna(0)
        return X


class RollingStatsAdder(BaseEstimator, TransformerMixin):
    def __init__(self, window=4):
        self.window = window

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.sort_values(['Store', 'Dept', 'Date']).copy()
        group = X.groupby(['Store', 'Dept'])

        X[f'RollingSalesMean_{self.window}'] = group['Weekly_Sales'].transform(
            lambda x: x.shift(1).rolling(self.window, min_periods=1).mean()
        ).fillna(0)

        X[f'RollingSalesStd_{self.window}'] = group['Weekly_Sales'].transform(
            lambda x: x.shift(1).rolling(self.window, min_periods=1).std()
        ).fillna(0)

        return X

class StoreAggregator(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.timeseries = {}

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        self.timeseries = {}
        for store in X['Store'].unique():
            self.aggregate_store_info(store, X)
        return self.timeseries

    def aggregate_store_info(self, store_id, X):
        store_data = X[X['Store'] == store_id].copy()

        # Check if Weekly_Sales exists (train data) or not (test data)
        has_weekly_sales = 'Weekly_Sales' in store_data.columns

        if has_weekly_sales:
            sum_columns = ['Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        else:
            sum_columns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']

        first_columns = ['IsHoliday', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
                        'Type', 'Size', 'Day', 'Month', 'Year', 'SuperbowlWeek',
                        'LaborDayWeek', 'ThanksgivingWeek', 'ChristmasWeek',
                        'Days_to_Thanksgiving', 'Days_to_Christmas']

        agg_dict = {}

        # Add sum columns that exist in the data
        for col in sum_columns:
            if col in store_data.columns:
                agg_dict[col] = 'sum'

        # Add first columns that exist in the data
        for col in first_columns:
            if col in store_data.columns:
                agg_dict[col] = 'first'

        aggregated = store_data.groupby(['Date', 'Store']).agg(agg_dict).reset_index()
        aggregated = aggregated.sort_values('Date').reset_index(drop=True)

        # Calculate department proportions only if Weekly_Sales exists
        if has_weekly_sales:
            dept_proportions = self.calculate_dept_proportions(store_data)
        else:
            dept_proportions = None

        self.timeseries[store_id] = (aggregated, dept_proportions)
        return aggregated

    def calculate_dept_proportions(self, store_data):
        dept_totals = store_data.groupby('Dept')['Weekly_Sales'].sum()
        store_total = store_data['Weekly_Sales'].sum()

        if store_total == 0:
            num_depts = len(dept_totals)
            return {dept: 1.0/num_depts for dept in dept_totals.index}

        dept_proportions = (dept_totals / store_total).to_dict()
        return dept_proportions

In [168]:
# Assuming df is your train.csv with 'Store' and 'Weekly_Sales'
train_dict = {}
for store_id, group in train.groupby("Store"):
    # You can define custom proportions here if needed
    train_dict[store_id] = (group.copy(), 1.0)


In [169]:
test_dict = {}

for store in test["Store"].unique():
    df_store = test[test["Store"] == store]
    dept = df_store["Dept"].iloc[0] if "Dept" in df_store.columns else 1
    test_dict[store] = (df_store, {"Dept": dept})


In [170]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('base_merge', BaseMerger(features, stores)),
    ('missing', MissingValueFiller()),
    ('categorical', CategoricalEncoder()),
    ('feature', FeatureAdder()),
    ('dept_prop', DeptProportionAdder()),
    ('lag_sales', LagSalesAdder()),
    ('rolling', RollingStatsAdder(window=4)),
    ('aggregator', StoreAggregator())
])


In [149]:
# # # import numpy as np
# # # import pandas as pd
# # # import mlflow
# # # import mlflow.lightgbm
# # # from lightgbm import LGBMRegressor

# # # def train_store_models_lgbm(train_weeks):
# # #     mape_array = []
# # #     wmae_array = []
# # #     store_models = {}
# # #     train_columns = {}

# # #     for store in train_dict.keys():
# # #         df, prop = train_dict[store]

# # #         df = df.sort_values("Date")

# # #         # Pipeline
# # #         df = BaseMerger(features, stores).fit_transform(df)
# # #         df = FeatureAdder(is_train=True).fit_transform(df)
# # #         df = MissingValueFiller(is_train=True).fit_transform(df)
# # #         df = CategoricalEncoder().fit_transform(df)

# # #         train_df = df.iloc[:train_weeks]
# # #         val_df = df.iloc[train_weeks:]

# # #         X_train = train_df.drop(columns=["Weekly_Sales", "Date"])

# # #         train_columns = X_train.columns

# # #         y_train = train_df["Weekly_Sales"]

# # #         # is_holiday_val = val_df["IsHoliday"]
# # #         X_val = val_df.drop(columns=["Weekly_Sales", "Date"])
# # #         y_val = val_df["Weekly_Sales"]

# # #         model = LGBMRegressor(n_estimators=800, learning_rate=0.1, max_depth=-1)

# # #         with mlflow.start_run(run_name=f"store_{store}"):
# # #             mlflow.log_params({
# # #                 "store": store,
# # #                 "n_estimators": 800,
# # #                 "learning_rate": 0.1,
# # #                 "max_depth": -1,
# # #             })

# # #             model.fit(X_train, y_train)

# # #             preds = model.predict(X_val)

# # #             # MAPE
# # #             mape = np.mean(np.abs((y_val - preds) / y_val)) * 100
# # #             mape_array.append(mape)

# # #             # WMAE
# # #             if "IsHoliday" in val_df.columns:
# # #                 weights = np.where(val_df["IsHoliday"] == 1, 5, 1)
# # #                 print(weights)
# # #             else:
# # #                 weights = np.ones(len(val_df))  # fallback equal weights

# # #             wmae = np.sum(weights * np.abs(y_val - preds)) / np.sum(weights)
# # #             wmae_array.append(wmae)

# # #             mlflow.log_metrics({
# # #                 "MAPE": mape,
# # #                 "WMAE": wmae
# # #             })

# # #             # Log model
# # #             # mlflow.lightgbm.log_model(model, artifact_path="model")

# # #             store_models[store] = (model, prop)

# # #     return mape_array, wmae_array, store_models, train_columns


# # def wmae_eval(y_pred, dataset):
# #     y_true = dataset.get_label()
# #     weights = dataset.get_weight()
# #     if weights is None:
# #         weights = np.ones_like(y_true)

# #     error = np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)
# #     return 'wmae', error, False  # False = lower is better

# # from lightgbm import LGBMRegressor
# # from sklearn.model_selection import train_test_split, RandomizedSearchCV

# # def train_store_models_lgbm(train_dict, features, stores, max_stores=10):
# #     store_models = {}
# #     train_columns = None

# #     param_grid = {
# #         "n_estimators": [100, 200, 300, 500],
# #         "learning_rate": [0.01, 0.05, 0.1],
# #         "max_depth": [3, 5, 7, 9],
# #         "min_child_weight": [1, 3, 5],
# #         "subsample": [0.7, 0.8, 1.0],
# #         "colsample_bytree": [0.7, 0.8, 1.0]
# #     }

# #     for i, (store, (df, prop)) in enumerate(train_dict.items()):
# #         if i >= max_stores:
# #             break

# #         df = df.sort_values("Date")
# #         df = BaseMerger(features, stores).fit_transform(df)
# #         df = FeatureAdder(is_train=True).fit_transform(df)
# #         df = MissingValueFiller(is_train=True).fit_transform(df)
# #         df = CategoricalEncoder().fit_transform(df)

# #         df["Weight"] = df["IsHoliday"].apply(lambda x: 5 if x else 1)

# #         X = df.drop(columns=["Weekly_Sales", "Date"])
# #         y = df["Weekly_Sales"]
# #         weights = df["Weight"]

# #         X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(
# #             X, y, weights, test_size=0.2, shuffle=False
# #         )

# #         search = RandomizedSearchCV(
# #             LGBMRegressor(),
# #             param_distributions=param_grid,
# #             n_iter=20,
# #             scoring='neg_mean_absolute_error',
# #             cv=3,
# #             verbose=0,
# #             n_jobs=-1,
# #             random_state=42
# #         )
# #         search.fit(X_train, y_train)

# #         best_model = search.best_estimator_

# #         best_model.fit(
# #             X_train, y_train,
# #             eval_set=[(X_val, y_val)],
# #             eval_metric="mae",
# #             # early_stopping_rounds=20,
# #             feval=wmae_eval,
# #             verbose=False
# #         )

# #         # Save model with metadata
# #         prop = {"Dept": df["Dept"].iloc[0]} if "Dept" in df.columns else {}
# #         store_models[store] = (best_model, prop)

# #         if train_columns is None:
# #             train_columns = X.columns  # capture column structure

# #     return store_models, train_columns

# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from lightgbm import LGBMRegressor
# import mlflow
# import mlflow.lightgbm

# def wmae_eval(y_pred, dataset):
#     y_true = dataset.get_label()
#     weights = dataset.get_weight()
#     if weights is None:
#         weights = np.ones_like(y_true)

#     error = np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)
#     return 'wmae', error, False  # False: lower is better

# def train_store_models_lgbm(train_dict, features, stores, max_stores=10):
#     store_models = {}
#     train_columns = None

#     param_grid = {
#         "n_estimators": [100, 300, 500],
#         "learning_rate": [0.01, 0.05, 0.1],
#         "max_depth": [3, 5, 7],
#         "min_child_weight": [1, 3],
#         "subsample": [0.7, 0.8, 1.0],
#         "colsample_bytree": [0.7, 0.8, 1.0]
#     }

#     for i, (store, (df, prop)) in enumerate(train_dict.items()):
#         if i >= max_stores:
#             break

#         df = df.sort_values("Date")

#         # Preprocessing pipeline
#         df = BaseMerger(features, stores).fit_transform(df)
#         df = FeatureAdder(is_train=True).fit_transform(df)
#         df = MissingValueFiller(is_train=True).fit_transform(df)
#         df = CategoricalEncoder().fit_transform(df)

#         # Ensure 'IsHoliday' exists
#         if "IsHoliday" not in df.columns:
#             df["IsHoliday"] = 0  # Default to not a holiday

#         df["Weight"] = df["IsHoliday"].apply(lambda x: 5 if x else 1)

#         # Prepare features/target
#         X = df.drop(columns=["Weekly_Sales", "Date"])
#         y = df["Weekly_Sales"]
#         weights = df["Weight"]

#         X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(
#             X, y, weights, test_size=0.2, shuffle=False
#         )

#         # Hyperparameter tuning
#         search = RandomizedSearchCV(
#             LGBMRegressor(),
#             param_distributions=param_grid,
#             n_iter=15,
#             scoring='neg_mean_absolute_error',
#             cv=3,
#             verbose=0,
#             n_jobs=-1,
#             random_state=42
#         )
#         search.fit(X_train, y_train)

#         best_model = search.best_estimator_

#         # Fit with WMAE eval metric
#         best_model.fit(
#             X_train, y_train,
#             sample_weight=w_train,
#             eval_set=[(X_val, y_val)],
#             eval_sample_weight=[w_val],
#             eval_metric="mae",
#         )

#         with mlflow.start_run(run_name=f"store_{store}"):
#             mlflow.log_params(best_model.get_params())
#             preds = best_model.predict(X_val)
#             wmae = np.sum(w_val * np.abs(y_val - preds)) / np.sum(w_val)
#             mlflow.log_metric("WMAE", wmae)
#             # mlflow.lightgbm.log_model(best_model, "model")

#         store_models[store] = (best_model, prop)

#         if train_columns is None:
#             train_columns = X.columns  # store once

#     return store_models, train_columns



In [158]:
# # 2. Train the model
# train_weeks = 120
# store_models, train_columns = train_store_models_lgbm(train_dict, features, stores, max_stores=len(train_dict))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001162 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2395
[LightGBM] [Info] Number of data points in the train set: 8030, number of used features: 26
[LightGBM] [Info] Start training from score 20703.954442
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001170 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2395
[LightGBM] [Info] Number of data points in the train set: 8030, number of used features: 26
[LightGBM] [In

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
🏃 View run store_1 at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0/runs/464820461d1d49c0bbf2a3eae92ae858
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001723 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2393
[LightGBM] [Info] Number of data points in the train set: 8025, number of used features: 26
[LightGBM] [Info] Start training from score 25814.261542
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

KeyboardInterrupt: 

In [151]:
# # # submission = []

# # # for store, (model, prop) in store_models.items():
# # #     df, _ = train_dict[store]

# # #     df = df.sort_values("Date")

# # #     # Preprocess test data
# # #     df = BaseMerger(features, stores).fit_transform(df)
# # #     df = FeatureAdder(is_train=True).fit_transform(df)
# # #     df = MissingValueFiller(is_train=True).fit_transform(df)
# # #     df = CategoricalEncoder().fit_transform(df)

# # #     val_df = df.iloc[train_weeks:]

# # #     X_val = val_df.drop(columns=["Weekly_Sales", "Date"])
# # #     dates = val_df["Date"]

# # #     preds = model.predict(X_val)

# # #     for idx, (i, row) in enumerate(val_df.iterrows()):
# # #         dept = prop["Dept"] if isinstance(prop, dict) and "Dept" in prop else 1
# # #         prediction_date = row["Date"]
# # #         submission.append({
# # #             "Id": f"{store}_{dept}_{prediction_date.strftime('%Y-%m-%d')}",
# # #             "Weekly_Sales": preds[idx]
# # #         })

# # # submission_df = pd.DataFrame(submission)
# # # submission_df.to_csv("submission_lgbm.csv", index=False)


# # predictions = {}

# # test["Date"] = pd.to_datetime(test["Date"])

# # # Group full test_df by store
# # for store, group in test.groupby("Store"):
# #     if store not in store_models:
# #         continue  # Skip if model not trained

# #     model, prop = store_models[store]

# #     group = group.sort_values("Date")

# #     # Preprocessing
# #     group = BaseMerger(features, stores).fit_transform(group)
# #     group = FeatureAdder(is_train=False).fit_transform(group)
# #     group = MissingValueFiller(is_train=False).fit_transform(group)
# #     group = CategoricalEncoder().fit_transform(group)

# #     X_test = group.drop(columns=["Date"])
# #     X_test = X_test.reindex(columns=train_columns, fill_value=0)

# #     preds = model.predict(X_test)

# #     for idx, (_, row) in enumerate(group.iterrows()):
# #         dept = row["Dept"]
# #         date = row["Date"]
# #         predictions[(store, dept, date)] = preds[idx]

# # submission_df = pd.DataFrame([
# #     {
# #         "Id": f"{row['Store']}_{row['Dept']}_{row['Date'].strftime('%Y-%m-%d')}",
# #         "Weekly_Sales": predictions.get((row['Store'], row['Dept'], row['Date']), 0.0)
# #     }
# #     for _, row in test.iterrows()
# # ])

# # print("Submission shape:", submission_df.shape)
# # submission_df.to_csv("submission_lgbm.csv", index=False)

# submission = []
# predictions = {}

# for store, (model, prop) in store_models.items():
#     df_test, _ = test_dict[store]

#     df_test = df_test.sort_values("Date")
#     df_test["Date"] = pd.to_datetime(df_test["Date"])

#     df_test = BaseMerger(features, stores).fit_transform(df_test)
#     df_test = FeatureAdder(is_train=False).fit_transform(df_test)
#     df_test = MissingValueFiller(is_train=False).fit_transform(df_test)
#     df_test = CategoricalEncoder().fit_transform(df_test)

#     X_test = df_test.drop(columns=["Date"])
#     X_test = X_test.reindex(columns=train_columns, fill_value=0)  # Ensure matching features

#     preds = model.predict(X_test)

#     for idx, row in df_test.iterrows():
#         dept = prop["Dept"] if isinstance(prop, dict) and "Dept" in prop else 1
#         date = row["Date"]
#         predictions[(store, dept, date)] = preds[idx]

# # Format as required
# test_df = pd.concat([test_dict[store][0] for store in test_dict])
# test_df["Date"] = pd.to_datetime(test_df["Date"])

# submission_df = pd.DataFrame([
#     {
#         "Id": f"{row['Store']}_{row['Dept']}_{row['Date'].strftime('%Y-%m-%d')}",
#         "Weekly_Sales": predictions.get((row["Store"], row["Dept"], row["Date"]), 0.0)
#     }
#     for _, row in test_df.iterrows()
# ])

# print("Submission shape:", submission_df.shape)
# submission_df.to_csv("submission_lgbm.csv", index=False)



Submission shape: (115064, 2)


In [171]:
from lightgbm import LGBMRegressor

def train_store_models_lgbm(train_dict, features_df, stores_df, max_stores=None):
    store_models = {}
    train_columns_set = set()

    for i, (store_id, (X_store, dept_props)) in enumerate(train_dict.items()):
        if max_stores is not None and i >= max_stores:
            break

        print(f"Training model for Store {store_id}...")

        X = X_store.drop(columns=["Weekly_Sales", "Date", "Store"], errors="ignore")
        y = X_store["Weekly_Sales"]

        model = LGBMRegressor(
            n_estimators=1000,
            learning_rate=0.03,
            num_leaves=31,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=42
        )

        model.fit(X, y)
        store_models[store_id] = model
        train_columns_set.update(X.columns)

    return store_models, list(train_columns_set)


In [172]:
import pandas as pd

def make_submission(test_df, test_dict, store_models, train_columns, submission_template_path, output_path):
    submission = pd.read_csv(submission_template_path)
    predictions = []

    for store_id, (X_store, dept_props) in test_dict.items():
        if store_id not in store_models:
            print(f"Skipping store {store_id}, no trained model.")
            continue

        model = store_models[store_id]
        X = X_store.drop(columns=["Date", "Store"], errors="ignore").copy()

        # Align columns
        for col in train_columns:
            if col not in X.columns:
                X[col] = 0
        X = X[train_columns]

        preds = model.predict(X)

        for idx, row in X_store.iterrows():
            date_str = row["Date"].strftime("%Y-%m-%d")
            if dept_props is not None:
                for dept, prop in dept_props.items():
                    predictions.append({
                        "Id": f"{int(row['Store'])}_{int(dept)}_{date_str}",
                        "Weekly_Sales": max(preds[idx] * prop, 0)  # clip negative
                    })
            else:
                predictions.append({
                    "Id": f"{int(row['Store'])}_{int(row['Dept'])}_{date_str}",
                    "Weekly_Sales": max(preds[idx], 0)
                })

    pred_df = pd.DataFrame(predictions)
    final_submission = submission[["Id"]].merge(pred_df, on="Id", how="left")
    final_submission["Weekly_Sales"] = final_submission["Weekly_Sales"].fillna(0)
    final_submission.to_csv(output_path, index=False)
    print(f"✅ Submission saved to {output_path}")


In [173]:
# Apply pipeline
X_train_transformed = pipeline.fit_transform(train)
X_test_transformed = pipeline.transform(test)

# Aggregate
aggregator = StoreAggregator()
train_dict = aggregator.fit_transform(X_train_transformed)
test_dict = aggregator.transform(X_test_transformed)

# Train
store_models, train_columns = train_store_models_lgbm(train_dict, features, stores)

# Predict and generate submission
make_submission(
    test_df=test,
    test_dict=test_dict,
    store_models=store_models,
    train_columns=train_columns,
    submission_template_path='sample_submission.csv',
    output_path='submission.csv'
)


KeyError: 'Column not found: Weekly_Sales'

In [ ]:
from google.colab import files
files.download('submission_lgbm.csv')